# Cosas por agregar
- visualización
- filtrado
- incluir en loc y iloc ejemplos temporales y booleanos (ej dfl.loc['20130102':'20130104'])

# Links de interés
- Datasets https://www.kaggle.com/datasets
- Documentación de Pandas https://pandas.pydata.org/docs/index.html 
- Cookbook de pandas https://pandas.pydata.org/docs/user_guide/cookbook.html
- Estructura y un par de cosas sobre data engineering https://apmonitor.com/pds/index.php/Main/DataPreparation

# Import importantes y cargo pickles de dfs

In [2]:
import kagglehub
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
import pickle


# Download latest version
kagglehub.dataset_download("isathyam31/adult-income-prediction-classification")


/home/capitanespiral/Documents/GitHub/data_science/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/home/capitanespiral/.cache/kagglehub/datasets/isathyam31/adult-income-prediction-classification/versions/1'

In [3]:
#Cargo pickles
with open("df.pkl","rb") as file:
    df=pickle.load(file)
    
with open("df_t.pkl","rb") as file:
    df_t=pickle.load(file)

# Recolectar data

## Datos en general

La función más usada, lejos es `read_csv` de pandas, donde basta que demos el path del archivo y su nombre, y se leerá si no hay mayor problema. Por default, la primera fila se transforma en las columnas del dataframe, que podemos ver usando `dataframe.columns`

In [ ]:
path="/home/capitanespiral/Documents/GitHub/data_science" #Aquí introducir tu path
filename="data.csv"

df=pd.read_csv(f"{path}/{filename}")
#Display entrega mucho mejor resultado que print para los dataframes!
display(df)
print(df.columns)

---
Opciones más usadas de `read_csv`:
- **sep o delimiter** &rarr; El separador de la data, usualmente se encuentra solo pero a veces es mejor darlo explícitamente. (en el caso que se confunda el identificador automático)
- **header** &rarr; Número de fila que se toma como nombre de columnas y desde el cual comienza la data, por default, cero (inicio del .csv).
- **skiprows** &rarr; Filas a saltar desde el inicio del archivo (indexeando desde cero, se puede entregar lista o tupla)
- **skipfooter** &rarr; Filas a saltar desde el final del archivo.
- **usecols** &rarr; Subset de columnas a usar (secuencia de números o nombres explícitos).

In [ ]:
#Si la primera fila está mala, cambiamos el header
df1=pd.read_csv("data_bad_first_row.csv") #Salió pésimo
df2=pd.read_csv("data_bad_first_row.csv",header=1) #Mucho mejor

display(df1)
display(df2)

In [ ]:
#Si la segunda está mala
df3=pd.read_csv("data_bad_second_row.csv") #Primer dato pésimo
df4=pd.read_csv("data_bad_second_row.csv",skiprows=[1],header=0) #Nos saltamos la segunda fila y conservamos el header

display(df3)
display(df4)

In [ ]:
#Seleccionemos ciertas columnas no más
df5=pd.read_csv("data_bad_first_row.csv",header=1,usecols=[0,3]) #Puede ser con números, aquí la primera y la cuarta
df6=pd.read_csv("data_bad_first_row.csv",header=1,usecols=range(4)) #Puede ser con iteradores, acá me entrega de la primera A la cuarta
df7=pd.read_csv("data_bad_first_row.csv",header=1,usecols=["age","education","workclass"]) #Puede ser con los nombres explícitos

display(df5)
display(df6)
display(df7)

In [ ]:
del df1,df2,df3,df4,df5,df6,df7

## Series de Tiempo

Para el caso de series de tiempo, hay que saber trabajar con `Timestamps`, `Datetime`, `Timedelta`, etc...

### Funciones generales 

- **pd.to_datetime()** &rarr; Transforma lo que le entregues en un "Datetime", funciona aceptando varios formatos y si le entregas secuencias.
- **pd.date_range()** &rarr; Para crear puntos en el tiempo equiespaceados, con un "start" un "end", posibilidad de "periods" o "freq"
    - **freq** más comunes:"y" (años),"m" (meses),"W" (semana), "B" (business day), "d" (días), "h" (horas), "min" (minutos), "s" (segundos) Ojo que hay *infinitas* opciones! revisar en https://pandas.pydata.org/docs/user_guide/timeseries.html#dateoffset-objects
- **df.resample().func()** &rarr; Entregando como variable una frecuencia, podemos "resamplear" según la función "func"

Ejemplos de estas funciones:

In [ ]:
##datetime con varios formatos
dt1=pd.to_datetime(["13/1/2018", np.datetime64("2018-01-13"), datetime.datetime(2018, 1, 13)],dayfirst=True)
print("dt1:",dt1,"\n")

In [ ]:
##date_range
#Creando una lista de tiempo cada tres horas
t1="1-1-2000"
t2="2000-3-10"
dt2=pd.date_range(start=t1,end=t2,freq="3h")
print("dt2:",dt2,"\n")

#Lo mismo pero con 15 minutos
t1="1-1-2000"
t2="2000-3-10"
dt3=pd.date_range(start=t1,end=t2,freq="15min")
print("dt3:",dt3,"\n")

#Si tengo periodos y frecuencia
dt4=pd.date_range(start="2018-8-1", periods=5, freq="2d")
print("dt4:",dt4,"\n")

In [ ]:
#Como usar el resampleo
idx = pd.date_range("2018-01-01", periods=10, freq="h") #ojo que 1h = h
ts = pd.Series(range(len(idx)), index=idx)
print("ts",ts,"\n",sep="\n")
#Downsample (también muy usado el sum)
ts_downsampled=ts.resample("2h").mean() #LEJOS el más útil
print("ts_downsampled",ts_downsampled,"\n",sep="\n")
#upsample (también muy usado bfill)
ts_upsampled=ts.resample("30min").ffill() #Sería interesante, acá interpolar
print("ts_upsampled",ts_upsampled,"\n",sep="\n")

In [16]:
del dt1,dt2,dt3,dt4,idx,ts,ts_upsampled,ts_downsampled

Ahora trabajemos con data de verdad:

In [ ]:
df_t=pd.read_csv("biomet1.csv",skiprows=[1],header=0) #Tenemos caso de segunda linea sin sentido
df_t["time_t"]=pd.to_datetime(df_t['date']+" "+df_t['time']) #Lo guardamos en una nueva columna
display(df_t)
print("\nLa columna nueva:",df_t["time_t"],sep="\n")

Nos servirá ahora
- **datetime.min() y .max()** &rarr; Evidente
- **datetimeindex.atributos** &rarr; nos permite acceder a muchas funciones de tiempo como "hour","day","minute","second","dayofyear", "strftime", entre otras (si es una serie, no un índice, agregar dt). La totalidad de atributos y métodos en https://pandas.pydata.org/docs/reference/api/pandas.DatetimeIndex.html. La totalidad de formatos para "strftime" en https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [ ]:
#Podemos acceder a propiedades o métodos específicos de un objeto tipo "datetime" usando el "dt"
print("Solo el año:",df_t["time_t"].dt.year,"\n",sep="\n")
print("Cambio el formato:",df_t["time_t"].dt.strftime("%Y  /   %m   /  %d"),"\n",sep="\n")

In [ ]:
#Fijamos un nuevo índice
df_t=df_t.set_index("time_t") #Esto "traslada" la columna "time_t" (ya no existe como columna) - Y pasa a ser datetimeindex
df_t=df_t.drop(columns = ["date","time"]) #Boto las columnas que ya no necesito
display(df_t)
print("Y el nuevo índice:")
print(df_t.index)

In [ ]:
#Al fijarlo como índice, ahora podemos accesar simplemente con ".atributo"
display(df_t)
print("Solo el día:",df_t.index.day,"\n",sep="\n")
print("Solo el día juliano:",df_t.index.dayofyear,"\n",sep="\n")

In [ ]:
#Podemos notar que hay vaciós en la data, en específico 3!
fig,ax=plt.subplots(figsize=(16,10))
ax.scatter(df_t.index,df_t.index,s=5)
plt.show()

In [ ]:
#Ahora cubrimos todo el espacio
ti=df_t.index.min()
tf=df_t.index.max()
time_total=pd.date_range(start=ti,end=tf,freq="30min")
df_t=df_t.reindex(time_total) #Esta función redefine el índice, conservando datos y agregando nans si el nuevo índice coincide con el anterior o si no existe (para cada fila!)
display(df_t)

In [ ]:
#Se rellenó
fig,ax=plt.subplots(figsize=(16,10))
ax.scatter(df_t.index,df_t.index,s=5)
plt.show()

## Almacenamos como pickle los dataframes

In [20]:
#El primer dataframe
with open('df.pkl',"wb") as file:
    pickle.dump(df,file)

#El dataframe de serie de tiempo
with open('df_t.pkl',"wb") as file:
    pickle.dump(df_t,file)

# Var data y estadística básica

## Indexación básica

Primero entender que *pandas* tiene dos grandes tipos de objetos: `Series`y `DataFrame`. El primero es como un "vector" (unidimensional) y el segundo está compuesto por varias `Series`, siendo como una "matriz". Y cada uno siempre tiene asociado su `Index`, que puede ser numérico o de cualquier naturaleza (uno común, temporal).

La forma básica de indexar cada tipo tiene distintos resultados:
- **Series[label]** &rarr; Dará un valor escalar en tal posición acorde al `Index`.
- **DataFrame[colname]** &rarr; Dará una *Serie* correspondiente a tal nombre de columna

In [3]:
#Accedo a una columna, me entrega a una serie, y despues a un elemento en específico
print("**Dataframe**")
print("Edad",df["age"],"\n",sep="\n")
print("Tercer valor:",df["age"][2],"\n") #df["age"] es una serie, entonces df["age"][2] es el valor asociado al indice 2 de la serie

#Si el índice no es numérico, esto igual se puede hacer, pero se recomienda indexar acorde al índice
print("**Dataframe temporal**")
print("Calor sensible",df_t["H"],"\n",sep="\n") #Notar que el índice es temporal
print("Tercer valor indexando con 'int':",df_t["H"][2]) #Obtengo el tercer valor -> Notar advertencia de pandas!
print("Tercer valor indexando con un 'label' del 'index':",df_t["H"]["2014-01-01 01:30:00"]) #Lo mismo pero con el tiempo, engorroso pero preciso

Siempre podemos acceder a **varios valores** de una *serie* (con indexación típica de python) o **varias columnas** en un *dataframe* (agregando un nuevo []:). Notar que esto último entrega un nuevo `DataFrame`.

In [4]:
display(df[["age","education"]])
display(df[["age","education"]][1:5]) 
print("Del segundo al quinto valor de una columna:",df["age"][1:5],sep="\n")

También usando slices de enteros en una *serie* o *dataframe* (en este caso se accesarán filas, no columnas)

In [ ]:
#Indexado clásico de python sobre una serie
print("Del primer al quinto dato:",df["age"][:5],"\n",sep="\n") 
print("Los datos pares:",df["age"][::2],"\n",sep="\n")

#Indexado clásico de python sobre un dataframe
print("\nDel primer al quinto dato:")
display(df_t[:5]) 
print("\nLos datos dados vuelta:")
display(df_t[::-1])

Para seleccionar algunas posiciones basta con entregar una lista, pero solo funciona con series!

In [10]:
print("Segundo, tercer y sexto dato de 'age':")
print(df["age"][[1,2,5]])

print("\nSegundo, tercer y sexto dato de 'H':")
df_t["H"][[1,2,5]] #Notar el warning

Segundo, tercer y sexto dato de 'age':
1    50
2    38
5    37
Name: age, dtype: int64

Segundo, tercer y sexto dato de 'H':


/tmp/ipykernel_8185/144011600.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_t["H"][[1,2,5]] #Notar el warning


2014-01-01 01:00:00    -4.62751
2014-01-01 01:30:00   -25.06920
2014-01-01 03:00:00   -13.89090
Name: H, dtype: float64

## Indexar con .loc e .iloc

Para indexar también se tienen los métodos `.loc` y `.iloc`(para *series* y *dataframes*)
1. `.loc` &rarr; trabaja con labels del índice o con arreglos booleanos.
2. `.iloc` &rarr; trabaja con posisiones con enteros (desde 0 a -1) o con arreglos booleanos.

In [ ]:
df_temp=df.drop([1]) #Boto la segunda fila
display(df_temp)

print("Con loc",df_temp.loc[2],"\n",sep="\n") #Aquí 2 es interpretado como "label", será la nueva segunda fila

print("Con loc",df_temp.iloc[2],"\n",sep="\n") #Aquí como entero, será la tercera fila

Para llamar a varios elementos no secuenciales, usamos también listas

In [ ]:
#Usando listas de elementos
print("Usando listas:")

display(df_temp.loc[[0,3,5]]) #El label 0, 3, 5

display(df_temp.iloc[[0,3,5]]) #Las posiciones asociados (otro label)

#También usando "slices"
print("\nUsando slices ahora:")

display(df_temp.loc[0:5])

display(df_temp.iloc[0:5])

print("En este caso funcionan igual por la naturaleza de este índice. NOTAR que se incluye el PRINCIPIO Y EL FINAL!")

Llamando distintas filas y columnas usando `.loc`

In [ ]:
#Mezclando filas y columnas loc
print("Slice y columna - loc",df_temp.loc[2:9,"age"],"\n",sep="\n")

print("Slice y slice - loc")
display(df_temp.loc[2:9,"age":"sex"])

print("\nFilas y columnas - loc")
display(df_temp.loc[[3,5,7],["age","country","race"]])

Llamando distintas filas y columnas usando `.iloc`

In [ ]:
#Mezclando filas y columnas iloc

print("Slice y columna - iloc",df_temp.iloc[2:9,0],"\n",sep="\n")

print("Slice y slice - iloc")
display(df_temp.iloc[2:9,0:7])

print("\nFilas y columnas - iloc")
display(df_temp.iloc[[3,5,7],[0,5,7]])

Usando slices en `.loc` siempre tiene que ser algo compatible con el índice del df (o al menos *transformable* en este!)

In [ ]:
#Con loc podemos usar slices compatibles con el tipo de índice
display(df_t.loc["20150112":"20160101"])
display(df_t.loc["2015":"2017"])
display(df_t.loc["2015"]) #Todo lo que tenga 2015!

Mezclando posición y label indexing:

In [23]:
#Con loc, en el índice de las filas podemos indexar posicionalmente el índice:
display(df.loc[df.index[[0,2,5]],["age","education"]])
display(df_t.loc[df_t.index[[0,2,5]],["H","LE"]])

#Con iloc, es un poco más enredado, pero podemos llamar el label de la columna:
display(df.iloc[[0,2,5],df.columns.get_indexer(["sex"])])
display(df_t.iloc[[0,2,5],df_t.columns.get_indexer(["H","LE"])])

,age,education
0,39,Bachelors
2,38,HS-grad
5,37,Masters


,H,LE
2014-01-01 00:30:00,2.1082,0.987573
2014-01-01 01:30:00,-25.0692,-0.209549
2014-01-01 03:00:00,-13.8909,0.025752


,sex
0,Male
2,Male
5,Female


,H,LE
2014-01-01 00:30:00,2.1082,0.987573
2014-01-01 01:30:00,-25.0692,-0.209549
2014-01-01 03:00:00,-13.8909,0.025752


Como vemos, indexado básico y `.loc` e `.iloc` gestionan *múltiples casos* de indexado. Si queremos solo conseguir *un* valor, se recomiendo usar `.at`y `.iat` (mucho más veloces para esto). Uno ve labels y el otro posiciones respectivamente.

In [31]:
print(df.at[2,"age"])
print(df.iat[2,0])

print(df_t.at[df_t.index[0],"H"])
print(df_t.iat[0,25])

38
38
2.1082
2.1082


## Indexar con booleanos

Dentro de las cosas más poderosas y usadas en *Pandas*. Los operadores entre comparaciones son `|` (or), `&` (and), `~` (not) y se deben agrupar con parentesis.

Si estamos trabajando con `series` se trabaja igual que un arreglo numpy:

In [37]:
#Con series
s=df["age"]
print("Edades menores a 30:")
print(s[s<30]) #Claro ques era lo mismo usar df["age"][df["age"]<30]

print("\nEdades menores a 30 o mayor/igual a 40:")
print(s[(s<30) | (s>= 40)])

print("\nEdades entre 30 y 35:")
print(s[(s>=30) & (s<=35)])

Edades menores a 30:
4        28
12       23
16       25
26       19
30       23
         ..
32529    29
32535    22
32555    22
32556    27
32559    22
Name: age, Length: 9711, dtype: int64

Edades menores a 30 o mayor/igual a 40:
1        50
3        53
4        28
6        49
7        52
         ..
32556    27
32557    40
32558    58
32559    22
32560    52
Name: age, Length: 23948, dtype: int64

Edades entre 30 y 35:
8        31
11       30
13       32
15       34
17       32
         ..
32536    34
32537    30
32544    31
32551    32
32553    32
Name: age, Length: 5214, dtype: int64


Si queremos trabajar con `Dataframes`

## Otras funciones de visualización

Hay una serie de funciones que nos permiten visualizar la data. Dentro de las más básicas:
- **df.head(n)** &rarr; nos permite ver las primeras "n" filas (sin n, por default 5), funciona para `Series` y `DataFrame`.
- **df.tail(n)** &rarr; nos permite ver las últimas "n" filas (lo mismo), funciona para `Series` y `DataFrame`.

In [11]:
#Para dataframes
display(df.head())
display(df.head(10))

#Para una serie
print(df["age"].head())

In [12]:
display(df.tail())

display(df["workclass"].tail())

## Estadística

Para tener una descripción estadística básica de nuestra data podemos usar `df.describe()`

In [ ]:
display(df.describe())
df_t.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,salary
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456,0.240810
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429,0.427581
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000,0.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000,0.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000,0.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000,0.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000,1.000000


,DOY,Ta,Pa,RH,Td,Tc,Rn,LWin,LWout,SWin,...,Ts_2_1_1,Ts_3_1_1,SWC_1_1_1,SWC_2_1_1,SWC_3_1_1,SHF_1_1_1,SHF_2_1_1,SHF_3_1_1,LE,H
count,65916.000000,52513.000000,53922.000000,47646.000000,49335.000000,52513.000000,47565.000000,47568.000000,47566.000000,47571.000000,...,5.830200e+04,5.724800e+04,57367.000000,57455.000000,57368.000000,59114.000000,59110.000000,59111.000000,5.251300e+04,52513.000000
mean,176.631558,283.168731,102175.206806,63.652512,3.808639,283.168731,77.835297,675.487559,734.376218,155.724590,...,-3.381937e+03,-5.324770e+12,0.562180,0.584705,0.311443,-0.125020,-0.350309,-0.474499,-1.223460e+02,82.933967
std,101.776559,21.535027,2498.673850,40.283867,21.686410,21.535027,203.979697,60.356268,47.074460,252.848755,...,1.566262e+05,9.372216e+13,0.125917,0.108636,0.143639,4.930163,3.141290,2.774658,2.257463e+04,2422.917355
min,1.000000,12.442400,88256.200000,-91.105500,-48.752500,12.442400,-153.013000,0.000000,-1925.630000,-7.686280,...,-1.757490e+07,-1.875950e+15,0.000000,0.000000,0.000000,-12.143900,-19.364500,-12.760700,-3.712310e+06,-267458.000000
25%,90.208200,281.976000,100687.000000,56.051725,-4.471790,281.976000,-46.579900,634.660750,704.915000,-1.723295,...,2.816330e+02,2.812100e+02,0.494216,0.544990,0.226947,-2.815517,-2.121848,-1.950870,-7.156740e-01,-21.851900
50%,176.031200,284.341000,101141.000000,80.993250,4.985470,284.341000,-2.475980,685.651000,731.302000,1.315270,...,2.833330e+02,2.829960e+02,0.568100,0.592789,0.300752,-0.110623,-0.199541,-0.295721,3.661330e+00,-2.183100
75%,261.854000,286.852000,101996.000000,92.223300,10.346250,286.852000,147.652000,720.179250,759.929750,231.949500,...,2.856600e+02,2.851710e+02,0.634363,0.639718,0.365071,2.192137,1.377710,1.040055,4.996030e+01,21.475800
max,366.979000,509.294000,109155.000000,105.718000,273.702000,509.294000,927.678000,1255.550000,1245.410000,1197.010000,...,3.449810e+02,3.278990e+02,1.048280,1.057090,0.799173,471.304000,401.488000,408.791000,4.249940e+05,120098.000000
